### Imports

In [1]:
%load_ext autoreload
%autoreload 2
import glob
import os
import random
import pathlib

import pandas as pd
import torch
from Bio.PDB.Polypeptide import index_to_one
from collections import OrderedDict
from torch.utils.data import DataLoader, Dataset

from cavity_model import (
    CavityModel,
    ResidueEnvironment,
    ResidueEnvironmentsDataset,
)
import helpers
from typing import Optional, List
from visualization import scatter_pred_vs_true, plot_validation_performance
import pickle

import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)

%load_ext nb_black

<IPython.core.display.Javascript object>

### Global variables

In [2]:
# Main parameters
WARM_START: Optional[List[str]] = [
    "cavity_models/repeat_00_model_epoch_01.pt",
    "cavity_models/repeat_01_model_epoch_02.pt",
    "cavity_models/repeat_02_model_epoch_01.pt",
    "cavity_models/repeat_03_model_epoch_02.pt",
    "cavity_models/repeat_04_model_epoch_02.pt",
    "cavity_models/repeat_05_model_epoch_02.pt",
]
DEVICE: str = "cuda"  # "cpu" or "cuda"
TRAIN_VAL_SPLIT: float = 0.9
BATCH_SIZE: int = 100
SHUFFLE_PDBS: bool = True
LEARNING_RATE: float = 2e-4
EPOCHS: int = 6
PATIENCE_CUTOFF: int = 0
EPS: float = 1e-9
N_MODEL_REPEATS = 6

# Parameters for simulation stride
STRIDE_FRAGMENTS: int = 2
STRIDE_MD: int = 4

# Parameters specific to downstream model
BATCH_SIZE_DDG: int = 40
SHUFFLE_DDG: bool = True
LEARNING_RATE_DDG: float = 1e-3
EPOCHS_DDG: int = 200

# Mapping between data keys and titles/colors for plots
dataset_name_mapping = {
    "dms": "DMS",
    "guerois": "Guerois",
    "protein_g": "Protein G",
    "symmetric_direct": "Symmetric (Direct)",
    "symmetric_inverse": "Symmetric (Inverse)",
}

dataset_color_mapping = {
    "dms": "steelblue",
    "guerois": "firebrick",
    "protein_g": "forestgreen",
    "symmetric_direct": "olive",
    "symmetric_inverse": "olive",
}

<IPython.core.display.Javascript object>

# Cavity Model

### Train

In [3]:
if WARM_START is None:
    # Create directory for model files
    models_dirpath = "cavity_models/"
    if not os.path.exists(models_dirpath):
        os.mkdir(models_dirpath)

    best_model_paths = []
    for i in range(N_MODEL_REPEATS):
        print(f"Repeat: {i:02d}")

        # Parse data
        parsed_pdb_filenames = sorted(glob.glob("data/pdbs/parsed/*coord*"))
        if SHUFFLE_PDBS:
            random.shuffle(parsed_pdb_filenames)
        (
            dataloader_train,
            dataset_train,
            dataloader_val,
            dataset_val,
        ) = helpers.train_val_split(
            parsed_pdb_filenames, TRAIN_VAL_SPLIT, DEVICE, BATCH_SIZE
        )

        # Define model
        cavity_model_net = CavityModel(DEVICE).to(DEVICE)
        loss_function = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(cavity_model_net.parameters(), lr=LEARNING_RATE)

        model_prefix = f"repeat_{i:02d}"
        # Train loop
        best_model_path = helpers.train_loop(
            dataloader_train,
            dataloader_val,
            cavity_model_net,
            loss_function,
            optimizer,
            model_prefix,
            EPOCHS,
            PATIENCE_CUTOFF,
        )

        best_model_paths.append(best_model_path)
else:
    print(f"Warm start: {WARM_START}")

Warm start: ['cavity_models/repeat_00_model_epoch_01.pt', 'cavity_models/repeat_01_model_epoch_02.pt', 'cavity_models/repeat_02_model_epoch_01.pt', 'cavity_models/repeat_03_model_epoch_02.pt', 'cavity_models/repeat_04_model_epoch_02.pt', 'cavity_models/repeat_05_model_epoch_02.pt']


<IPython.core.display.Javascript object>

# Infer ddGs

### Load structural data for ddG data sets

In [4]:
# Create temporary residue environment datasets as dicts to more easily match ddG data
parsed_pdbs_wildcards = {
    "dms": "data/data_dms/pdbs_parsed/*coord*",
    "protein_g": "data/data_protein_g/pdbs_parsed/*coord*",
    "guerois": "data/data_guerois/pdbs_parsed/*coord*",
    "symmetric": "data/data_symmetric/pdbs_parsed/*coord*",
}

resenv_datasets_look_up = {}
for dataset_key, pdbs_wildcard in parsed_pdbs_wildcards.items():
    parsed_pdb_filenames = sorted(glob.glob(pdbs_wildcard))
    dataset = ResidueEnvironmentsDataset(parsed_pdb_filenames, transformer=None)
    dataset_look_up = {}
    for resenv in dataset:
        key = (
            f"{resenv.pdb_id}{resenv.chain_id}_{resenv.pdb_residue_number}"
            f"{index_to_one(resenv.restype_index)}"
        )
        dataset_look_up[key] = resenv
    resenv_datasets_look_up[dataset_key] = dataset_look_up

<IPython.core.display.Javascript object>

### Load ddG data sets

In [5]:
ddg_data_dict = OrderedDict()
ddg_data_dict = {
    "dms": pd.read_csv("data/data_dms/ddgs_parsed.csv"),
    "protein_g": pd.read_csv("data/data_protein_g/ddgs_parsed.csv"),
    "guerois": pd.read_csv("data/data_guerois/ddgs_parsed.csv"),
    "symmetric_direct": pd.read_csv("data/data_symmetric/ddgs_parsed_direct.csv"),
    "symmetric_inverse": pd.read_csv("data/data_symmetric/ddgs_parsed_inverse.csv"),
}
# Load the residue environments to the dataframes
helpers.populate_dfs_with_resenvs(ddg_data_dict, resenv_datasets_look_up)

Dropping data points where residue is not defined in structure or due to missing parsed pdb file
dropped 1187 / 8096 data points from dataset dms
dropped    0 /  907 data points from dataset protein_g
dropped    0 /  911 data points from dataset guerois
dropped    0 /  342 data points from dataset symmetric_direct
dropped    0 /  342 data points from dataset symmetric_inverse


<IPython.core.display.Javascript object>

### Load cavity models

In [6]:
# Load best performing cavity model from previos training
if WARM_START is not None:
    best_model_paths = WARM_START

cavity_model_infer_nets = []
for best_path in best_model_paths:
    cavity_model_infer_net = CavityModel(DEVICE).to(DEVICE)
    cavity_model_infer_net.load_state_dict(torch.load(best_path))
    _ = cavity_model_infer_net.eval()
    cavity_model_infer_nets.append(cavity_model_infer_net)

<IPython.core.display.Javascript object>

# Process data

In [7]:
# Setting 1: Static setting
if not os.path.exists("processed_data_frames.pickle"):
    helpers.populate_dfs_with_nlls_and_nlfs(
        ddg_data_dict, cavity_model_infer_nets, DEVICE, BATCH_SIZE, EPS
    )

<IPython.core.display.Javascript object>

In [ ]:
# Setting 2: WT Structural ensemvle
if not os.path.exists("processed_data_frames.pickle"):
    # Infer NLLs for WT and MT and add to columns `wt_nll_md` and `mt_nll_md`
    helpers.infer_molecular_dynamics_nlls(
        ddg_data_dict,
        "protein_g",
        DEVICE,
        EPS,
        cavity_model_infer_nets,
        stride=STRIDE_MD,
        is_npt_ensemble=True,
        use_residue_number_map=True,
    )
    helpers.infer_molecular_dynamics_nlls(
        ddg_data_dict,
        "guerois",
        DEVICE,
        EPS,
        cavity_model_infer_nets,
        stride=STRIDE_MD,
        is_npt_ensemble=False,  # Maybe fix the bug with NPT ensemble
        use_residue_number_map=True,
    )
    helpers.infer_molecular_dynamics_nlls(
        ddg_data_dict,
        "dms",
        DEVICE,
        EPS,
        cavity_model_infer_nets,
        stride=STRIDE_MD,
        is_npt_ensemble=True,
        use_residue_number_map=False,
    )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566


608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857


898
899
900
901
902
903
904
905
906
907
908
909
910
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
26

In [ ]:
# Setting 3: Variant structures
if not os.path.exists("processed_data_frames.pickle"):
    helpers.get_predictions_both_structures(ddg_data_dict, len(cavity_model_infer_nets))

In [ ]:
# Setting 4: Modelling the unfolded state
if not os.path.exists("processed_data_frames.pickle"):
    for data_set in ["protein_g", "dms", "guerois"]:
        helpers.infer_probabilities_for_center_residues(
            ddg_data_dict,
            data_set,
            cavity_model_infer_nets,
            DEVICE,
            EPS,
            is_wt=True,
            stride=STRIDE_FRAGMENTS,
        )
        helpers.infer_probabilities_for_center_residues(
            ddg_data_dict,
            data_set,
            cavity_model_infer_nets,
            DEVICE,
            EPS,
            is_wt=False,
            stride=STRIDE_FRAGMENTS,
        )

        helpers.add_ddg_preds_with_unfolded_state(
            ddg_data_dict, data_set, len(cavity_model_infer_nets)
        )
        
    # Calculate DDgs
    helpers.add_ddg_preds_with_md_simulations(
        ddg_data_dict, "protein_g", len(cavity_model_infer_nets)
    )
    helpers.add_ddg_preds_with_md_simulations(
        ddg_data_dict, "guerois", len(cavity_model_infer_nets)
    )
    helpers.add_ddg_preds_with_md_simulations(
        ddg_data_dict, "dms", len(cavity_model_infer_nets)
    )

### Cache processed data

In [ ]:
if not os.path.exists("processed_data_frames.pickle"):
    with open("processed_data_frames.pickle", "wb") as handle:
        pickle.dump(ddg_data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Load processed data

In [ ]:
with open("processed_data_frames.pickle", "rb") as handle:
    ddg_data_dict = pickle.load(handle)

# Figures in for paper

In [ ]:
labels = ["DMS", "Protein G", "Guerois"]

cavity_correlations = [
    np.mean(ULTRA_VANILLA_RS["dms"]),
    np.mean(ULTRA_VANILLA_RS["protein_g"]),
    np.mean(ULTRA_VANILLA_RS["guerois"]),
]

cavity_stds = [
    np.std(ULTRA_VANILLA_RS["dms"]),
    np.std(ULTRA_VANILLA_RS["protein_g"]),
    np.std(ULTRA_VANILLA_RS["guerois"]),
]

rosetta_correlations = [0.5, 0.73, 0.71]
foldx_correlations = [0.31, 0.44, 0.66]

# Figure 3a
df_figure_3a = pd.DataFrame(
    {
        "data_set": labels,
        "cavity_correlations": cavity_correlations,
        "cavity_stds": cavity_stds,
        "rosetta_correlations": rosetta_correlations,
        "foldx_correlations": foldx_correlations,
    }
)

In [ ]:

economist_colors = ['#DB444B', '#006BA2', '#3EBCD2', '#379A8B', '#EBB434', '#B4BA39', '#9A607F', '#D1B07C']
#economist_colors = ['#EC111A', '#89ACC1', '#E38D8A']
saturation = 0.75

sns.set()
#colors = sns.color_palette('dark')
colors = economist_colors

sns.set(font_scale = 1.2)
sns.set_palette(sns.color_palette(colors))
sns.set_style("whitegrid")

def style_barplot_vertical(ax, legend_y, ncols=2):
    sns.despine(left=True, bottom=True, right=True)
    g.set(xlabel=None)
    g.get_legend().set_title(None)
    h,l = ax.get_legend_handles_labels()
    ax.legend_.remove()
    ax.legend(h,l, ncol=ncols, frameon=False, loc='upper right', bbox_to_anchor=(0.925,legend_y)) 
    ax.axhline(y=0, color='black')

def style_barplot(ax, legend_y, ticks_above=False, ncols=2):
    sns.despine(left=True, bottom=True, right=True)
    g.set(ylabel=None)
    g.get_legend().set_title(None)
    h,l = ax.get_legend_handles_labels()
    ax.legend_.remove()
    if ticks_above:
        ax.xaxis.set_label_position("top")
        ax.xaxis.labelpad=10 
        ax.tick_params(axis='x', which='major', labelbottom = False, bottom=False, top = False, labeltop=True, pad=-5)
        fig.legend(h,l, ncol=ncols, frameon=False, loc='lower right', bbox_to_anchor=(0.925,legend_y)) 
    else:
        ax.tick_params(axis='x', which='major', pad=-2)
        fig.legend(h,l, ncol=ncols, frameon=False, loc='lower right', bbox_to_anchor=(0.925,legend_y)) 

    ax.axvline(x=0, color='black')
    ax.set_xlim([0,0.79])


data = pd.read_csv('fig2.csv')
data = data.reindex([1,2,0])
data['data_set'].replace('DMS', 'VAMP-seq', inplace=True)

data = pd.concat([data[['data_set', 'cavity_correlations', 'cavity_stds']].rename(columns={'cavity_correlations':'correlations', 'cavity_stds':'std'}).assign(type='cavity'),
                  data[['data_set', 'rosetta_correlations']].rename(columns={'rosetta_correlations':'correlations'}).assign(type='rosetta'),
                  data[['data_set', 'foldx_correlations']].rename(columns={'foldx_correlations':'correlations'}).assign(type='foldx')])

fig, ax = plt.subplots(figsize=(5,3))
g = sns.barplot(data=data, y='data_set', x='correlations', hue='type', ci='std', ax=ax, edgecolor='black',linewidth=0, saturation=saturation)
mask = data["std"].notna().to_numpy()
#x_coords = np.array([p.get_x() + 0.5*p.get_width() for p in ax.patches])[mask]
#y_coords = np.array([p.get_height() for p in ax.patches])[mask]
y_coords = np.array([p.get_y() + 0.5*p.get_height() for p in ax.patches])[mask]
x_coords = np.array([p.get_width() for p in ax.patches])[mask]
ax.errorbar(x=x_coords, y=y_coords, xerr=data["std"][mask], color=sns.desaturate(sns.color_palette()[0], saturation), fmt="none", capsize=5)
g.set_xlabel("correlation")
#style_barplot(legend_y=-0.02, ncols=3, ticks_above=True)
style_barplot(ax=ax,legend_y=0.85, ncols=3, ticks_above=False)
data
plt.savefig('figure2.pdf', bbox_inches='tight')
